In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_09_optimizer")' FastaiNotebook_09_optimizer

Installing packages:
	.package(path: "/content/FastaiNotebook_09_optimizer")
		FastaiNotebook_09_optimizer
With SwiftPM flags: []
Working in: /tmp/tmp0o0ipo4o/swift-install
Updating https://github.com/mxcl/Path.swift
Updating https://github.com/saeta/Just
Updating https://github.com/latenitesoft/NotebookExport
Completed resolution in 2.81s
[1/15] Compiling FastaiNotebook_09_optimizer 00_load_data.swift
[2/15] Compiling FastaiNotebook_09_optimizer 01_matmul.swift
[3/15] Compiling FastaiNotebook_09_optimizer 01a_fastai_layers.swift
[4/15] Compiling FastaiNotebook_09_optimizer 02_fully_connected.swift
[5/15] Compiling FastaiNotebook_09_optimizer 02a_why_sqrt5.swift
[6/15] Compiling FastaiNotebook_09_optimizer 03_minibatch_training.swift
[7/15] Compiling FastaiNotebook_09_optimizer 04_callbacks.swift
[8/15] Compiling FastaiNotebook_09_optimizer 05_anneal.swift
[9/15] Compiling FastaiNotebook_09_optimizer 05b_early_stopping.swift
[10/15] Compiling FastaiNotebook_09_optimizer 06_cuda.swift
[

In [0]:
// export
import Path
import TensorFlow

In [0]:
import FastaiNotebook_09_optimizer

In [4]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')



Load data

//TODO: switch to imagenette when possible to train


In [5]:
let data = mnistDataBunch(flat: true)

2019-08-23 21:24:40.868398: W tensorflow/core/framework/allocator.cc:107] Allocation of 188160000 exceeds 10% of system memory.
2019-08-23 21:24:40.985066: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-08-23 21:24:40.997541: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-08-23 21:24:40.997910: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2db0a80 executing computations on platform Host. Devices:
2019-08-23 21:24:40.997973: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-08-23 21:24:41.047306: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-08-23 21:24:41.050210: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable

In [0]:
let (n,m) = (60000,784)
let c = 10
let nHid = 50

In [0]:
func modelInit() -> BasicModel {return BasicModel(nIn: m, nHid: nHid, nOut: c)}

In [0]:
let learner = Learner(data: data, lossFunc: softmaxCrossEntropy, optFunc: sgdOpt(lr: 0.1), modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [9]:
learner.fit(1)

Epoch 0: [0.37816286, 0.9114]


Mixup

In [0]:
//export
extension RandomDistribution {
    // Returns a batch of samples.
    func next<G: RandomNumberGenerator>(
        _ count: Int, using generator: inout G
    ) -> [Sample] {
        var result: [Sample] = []
        for _ in 0..<count {
            result.append(next(using: &generator))
        }
        return result
    }

    // Returns a batch of samples, using the global Threefry RNG.
    func next(_ count: Int) -> [Sample] {
        return next(count, using: &ThreefryRandomNumberGenerator.global)
    }
}

Mixup requires one-hot encoded targets since we don't have a loss function with no reduction.

In [0]:
//export
extension Learner {
    public class MixupDelegate: Delegate {
        private var distribution: BetaDistribution
        
        public init(alpha: Float = 0.4){
            distribution = BetaDistribution(alpha: alpha, beta: alpha)
        }
        
        override public func batchWillStart(learner: Learner) {
            if let xb = learner.currentInput {
                if let yb = learner.currentTarget as? Tensor<Float>{
                    var lambda = Tensor<Float>(distribution.next(Int(yb.shape[0])))
                    lambda = max(lambda, 1-lambda)
                    let shuffle = Raw.randomShuffle(value: Tensor<Int32>(0..<Int32(yb.shape[0])))
                    let xba = Raw.gather(params: xb, indices: shuffle)
                    let yba = Raw.gather(params: yb, indices: shuffle)
                    lambda = lambda.expandingShape(at: 1)
                    learner.currentInput = lambda * xb + (1-lambda) * xba
                    learner.currentTarget = (lambda * yb + (1-lambda) * yba) as? Label
                }
            }
        }
    }
    
    public func makeMixupDelegate(alpha: Float = 0.4) -> MixupDelegate {
        return MixupDelegate(alpha: alpha)
    }
}

In [0]:
let (n,m) = (60000,784)
let c = 10
let nHid = 50

We need to one-hot encode the targets

In [0]:
var train1 = data.train.innerDs.map { DataBatch<TF,TF>(xb: $0.xb, 
                            yb: Raw.oneHot(indices: $0.yb, depth: TI(10), onValue: TF(1), offValue: TF(0))) }

In [0]:
var valid1 = data.valid.innerDs.map { DataBatch<TF,TF>(xb: $0.xb, 
                            yb: Raw.oneHot(indices: $0.yb, depth: TI(10), onValue: TF(1), offValue: TF(0))) }

In [0]:
let data1 = DataBunch(train: train1, valid: valid1, trainLen: data.train.dsCount, 
                  validLen: data.valid.dsCount, bs: data.train.bs)

In [0]:
func modelInit() -> BasicModel {return BasicModel(nIn: m, nHid: nHid, nOut: c)}

In [0]:
func accuracyFloat(_ out: TF, _ targ: TF) -> TF {
    return TF(out.argmax(squeezingAxis: 1) .== targ.argmax(squeezingAxis: 1)).mean()
}

In [0]:
let learner = Learner(data: data1, lossFunc: softmaxCrossEntropy, optFunc: sgdOpt(lr: 0.1), modelInit: modelInit)
let recorder = learner.makeRecorder()

In [0]:
learner.delegates = [learner.makeTrainEvalDelegate(), learner.makeShowProgress(), 
                     learner.makeAvgMetric(metrics: [accuracyFloat]), recorder,
                     learner.makeMixupDelegate(alpha: 0.2)]

In [20]:
learner.fit(2)

Epoch 0: [0.5435329, 0.9135]
Epoch 1: [0.51660204, 0.9194]


Labelsmoothing

In [0]:
//export
@differentiable(wrt: out)
public func labelSmoothingCrossEntropy(_ out: TF, _ targ: TI, ε: Float = 0.1) -> TF {
    let c = out.shape[1]
    let loss = softmaxCrossEntropy(logits: out, labels: targ)
    let logPreds = logSoftmax(out)
    return (1-ε) * loss - (ε / Float(c)) * logPreds.mean()
}

In [0]:
@differentiable(wrt: out)
func lossFunc(_ out: TF, _ targ: TI) -> TF { return labelSmoothingCrossEntropy(out, targ, ε: 0.1) }

In [0]:
let learner = Learner(data: data, lossFunc: lossFunc, optFunc: sgdOpt(lr: 0.1), modelInit: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])
learner.delegates.append(learner.makeNormalize(mean: mnistStats.mean, std: mnistStats.std))

In [24]:
learner.fit(2)

Epoch 0: [0.2978364, 0.9329]
Epoch 1: [0.2847682, 0.94]


Export

In [25]:
import NotebookExport
let exporter = NotebookExport(Path.cwd/"10_mixup_ls.ipynb")
print(exporter.export(usingPrefix: "FastaiNotebook_"))

success
